In [68]:
from openai import OpenAI
from typing import List, Dict

def ask(messages: List[Dict], is_json:bool = False):
        client = OpenAI(api_key='sk-a6bb624707354861abff16885ffe5252', base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

        if is_json:
                response = client.chat.completions.create(
                        model = 'qwen-plus',
                        temperature = 0,
                        messages = messages,
                        response_format={ "type": "json_object" }
                )
        else:
                response = client.chat.completions.create(
                        model = 'qwen-plus',
                        temperature = 0,
                        messages = messages,
                )

        return response

In [69]:
## Web Search Tool
from search import internet_search, process_content

class WebSearch:
    def __init__(self, name:str='web_search', threhold:int=8000):
        self.system_prompt = """
You are a Insight Researcher.

1. To find detail informtion for the user query
and summary the content into one sentence as simple as possible
2. If the user's question is about specific numerical values, 
only return the numerical results without any additional explanation.
"""
        self.name = name
        self.description = "the tool use for web search"
        self.threhold = threhold

    def __call__(self, query:str):
        # print(query)
        results = internet_search(query)
        # print(results)
        all_text = ""
        windows_size = 0
        for item in results:
            if windows_size >= self.threhold:
                break
            page_content = process_content(item['href'])
            for page in page_content:
                if windows_size + len(page) > self.threhold:
                    remaining_space = self.threhold - windows_size
                    all_text += page[:remaining_space].strip() + "\n\n"
                    windows_size = self.threhold
                    break
                else:
                    windows_size += len(page)
                    all_text += page + "\n\n"
            if windows_size >= self.threhold:
                break

        msg = [{"role":"system","content":self.system_prompt},
               {"role":"user", "content": f"The search results are {all_text}"}]
        
        answer = ask(messages=msg)

        return answer.choices[0].message.content

In [70]:
#Calculation Tool
class AbsDifference:
    def __init__(self, name:str="subtractions"):
        self.system_prompt = "You are a calculation assistant."

        self.name = name
        self.description = "math tool for calculating subtractions"
        
    def __call__(self, result1:str, result2:str) -> str:
        question = f"To calculate the abs difference between {result1} and {result2}"

        user_content = f"""
Answer the Question based on the Context. When you write down a expression, it MUST ONLY consists of numbers and operators. Here are some guidelines that you will be PANALIZED if you don't follow:

  - When you are asked for differences, you consider the absolute value of the difference. Difference of two numbers is always positive.For instance, the difference between 1 and 2 is 1, not -1.
  - When you are applying operations (e.g. difference, summation, ratio, etc.) between multiple values in the Context, you must unify the units of those numbers. For instance, you cannot add 1 meter to 1 foot.
     - You must pick the values in the same units if all the values are available in the same units.
     - If not, you must convert them to the same units before applying the operation.
  - You MUST strictly follow the unit (e.g. meter, kilometer, million, etc.) you were asked.
     - If the Context has the numbers in same units as the question, you can directly use them.
     - If the Context has the numbers in different units than the question, you must convert them to the units asked in the question.For example, if the question asks for the distance between two cities in kilometers, but the Context has the distance in miles, you must convert the distance to kilometers.
  - If you are asked about a particular number in millions, billions, or any other unit, the number should be written without specifying the unit. For example, if you are asked for 100 millions, it should be written as 100, not 100 million or 100,000,000.
 - Never introduce a variable. For instance "gazelle_max_speed * 1.4" is not allowed. Pick up a correct number from the given context.

Here is an example of a correct answer:
If result1 is 100 and result2 is 50, you should output: |100 - 50| = 50.

 Question: {question}
 
"""
        messages = [
            {"role":"system", "content":self.system_prompt},
            {"role":"user", "content":user_content}
        ]

        response = ask(messages=messages, is_json=True)
        return response.choices[0].message.content

In [71]:
search_tool = WebSearch(name="search")
math_tool = AbsDifference(name="math")

tools = [search_tool, math_tool]

In [76]:
#制定计划流程和格式
system_prompt = f"""
Let's first understand the problem and devise a plan to solve the problem.
Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. 
Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, 
the final step should almost always be 'Given the above steps taken, please respond to the users original question'. 

If you can answer the question, please output'Thought: I can answer the question now.' at the end of the plan. 
Each plan should comprise an action from the following {len(tools)} types:
"""

for i, tool in enumerate(tools):
    system_prompt += f"{i+1}. {tool.name}: {tool.description}\n"

print(system_prompt)


Let's first understand the problem and devise a plan to solve the problem.
Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. 
Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, 
the final step should almost always be 'Given the above steps taken, please respond to the users original question'. 

If you can answer the question, please output'Thought: I can answer the question now.' at the end of the plan. 
Each plan should comprise an action from the following 2 types:
1. search: the tool use for web search
2. math: math tool for calculating subtractions



In [73]:
# 为了进一步提升系统的准确性，使用few-shot CoT的方法，训练模型能够快速理解用户的意图，并根据用户的意图生成相应的回复。
EXAMPLE_PROMPT = (
    "Question: If cheetah was 1.3 times slower, greyhound was 1.5 times faster, and falcon was 2.3 time slower then their maximum speeds, "
    "what will be the ratio of the fastest animal to the slowest animal?\n"
    '1. search("cheetah")\n'
    '2. math("cheetah max speed in km/h if 1.3 times slower?", ["$1"]\n'
    '3. search("greyhound")\n'
    '4. math("greyhound max speed in km/h if 1.5 times faster?", ["$3"]\n'
    '5. search("falcon")\n'
    '6. math("falcon max speed in km/h if 2.3 times slower?", ["$5"]\n'
    '7. math("max($2, $4, $6) / min($2, $4, $6)")\n'
    "Thought: I can answer the question now.\n"
    "###\n"
    "\n"
    "Question: If Mount Everest's height were halved and Mount Kilimanjaro's height were doubled, what would be the difference in their height?\n"
    "1. search('Mount Everest')\n"
    '2. math("half of Mount Everest height in meter?", ["$1"])\n'
    '3. search("Mount Kilimanjaro")\n'
    '4. math("double of Mount Kilimanjaro height in meter?", ["$3"])\n'
    '5. math("abs($3 - $4)")\n'
    "Thought: I can answer the question now.\n"
    "###\n"
    "\n"
    "Question: With the Sahara Desert's area reduced by 33% and the Kalahari Desert's area magnified by 52%, which one covers more ground?\n"
    "1. search('Sahara Desert')\n"
    "2. math('Sahara Desert area in km^2 if reduced by 33%?', ['$1'])\n"
    '3. search("Kalahari Desert")\n'
    "4. math('Kalahari Desert area in km^2 if magnified by 52%?', ['$3'])\n"
    "Thought: I can compare the numbers without calling math.\n"
    "###\n"
    "\n"
    "Question: Determine the smaller value: the depth difference in meters between the Mariana Trench and the Puerto Rico Trench, "
    "or the depth difference in meters between the South Sandwich Trench and the Sunda Trench.\n"
    "1. search('Mariana Trench')\n"
    "2. search('Puerto Rico Trench')\n"
    "3. math('absolute depth difference between Mariana and Puerto Rico Trench in meters?', ['$1', '$2'])\n"
    "4. search('South Sandwich Trench')\n"
    "5. search('Sunda Trench')\n"
    "6. math('absolute depth difference between South Sandwich and Sunda Trench in meters?', ['$4', '$5'])\n"
    "7. math('min($3, $6)')\n"
    "Thought: I can answer the question now.\n"
    "###\n"
    "\n"
    "Question: What is the raio of the height of Mount Everest and the height of Mount Kilimanjaro?\n"
    "1. search('Mount Everest')\n"
    "2. search('Mount Kilimanjaro')\n"
    "3. math('height of Mount Everest / height of Mount Kilimanjaro', ['$1', '$2'])\n"
    "Thought: I can answer the question now.\n"
    "###\n"
)
# the above prompt is refer to llm compiler

system_prompt = system_prompt + '\n' + EXAMPLE_PROMPT
print(system_prompt)


Let's first understand the problem and devise a plan to solve the problem.
Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. 
Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, 
the final step should almost always be 'Given the above steps taken, please respond to the users original question'. 

If you can answer the question, please output'Thought: I can answer the question now.' at the end of the plan. 
Each plan should comprise an action from the following 3 types:
1. search: the tool use for web search
2. math: math tool for calculating subtractions

Question: If cheetah was 1.3 times slower, greyhound was 1.5 times faster, and falcon was 2.3 time slower then their maximum speeds, what will be the ratio of the fastest animal to the slowest animal?
1. search("cheetah")
2. math("cheetah max speed in km/h if 1.3 times slower?", ["$1"]
3. search("greyhound")
4. math(

In [75]:
#步骤3中的["$1", "$2"]代表需要步骤1、2的结果作为输入。
user_content = "the currentpopulation gap between Toronto and New York city"

messages = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content":user_content}
]

response = ask(messages=messages)

plan = response.choices[0].message.content
print(plan)

Plan:
1. search("current population of Toronto")
2. search("current population of New York City")
3. math("population of New York City - population of Toronto", ["$1", "$2"])
Thought: I can answer the question now.
